In [1]:
%load_ext notexbook

In [2]:
%texify

# MTG Manager Collection Analysis

In this notebook, I will be playing with the exported data from my **MTGManager** collection (in `CSV` format) in order to identify the possible mismatch noticed after App migration.

## Preamble

In [3]:
import pandas as pd 
import os
from pathlib import Path

In [4]:
from mtg import DATA_FOLDER

MTG_MANAGER = DATA_FOLDER / "MTGManager" / "original"
MTG_DRAGONSHIELD = DATA_FOLDER / "MTGDragonShield" / "original"

In [5]:
from functools import partial 

def fullpath(path: str, base: Path) -> Path:
    return base / path

manager_path = partial(fullpath, base=MTG_MANAGER)
dshield_path = partial(fullpath, base=MTG_DRAGONSHIELD)

manager_files = list(map(manager_path, filter(lambda f: f.endswith(".csv"), os.listdir(MTG_MANAGER))))
dshield_files = list(map(dshield_path, filter(lambda f: f.endswith(".csv"), os.listdir(MTG_DRAGONSHIELD))))

Check the two lists are of the same length

In [6]:
len(manager_files) == len(dshield_files)

True

**Finally**, let's see whether `Collection` instances in the two (app) lists are `1-to-1` aligned:

First, let's verify that all filenames are indeed the same - so we are sure there is `1:1` matches for data files in the two App collections and we could proceed by _name_:

In [7]:
set(f.name for f in manager_files) == set(f.name for f in dshield_files)

True

Let's just have a quick look at these files:

In [8]:
app_name = lambda f: f.parent.parent.name
for i, (f1, f2) in enumerate(zip(manager_files, dshield_files)):
    print(f"{i+1:02d}): {app_name(f1)}/{f1.parent.name}/{f1.name} vs {app_name(f2)}/{f2.parent.name}/{f2.name}")

01): MTGManager/original/Black_Collection.csv vs MTGDragonShield/original/Black_Collection.csv
02): MTGManager/original/Non_basic_Lands.csv vs MTGDragonShield/original/Non_basic_Lands.csv
03): MTGManager/original/Artifacts.csv vs MTGDragonShield/original/Artifacts.csv
04): MTGManager/original/Green_Collection.csv vs MTGDragonShield/original/Green_Collection.csv
05): MTGManager/original/Islands.csv vs MTGDragonShield/original/Islands.csv
06): MTGManager/original/Gold_Multicolor.csv vs MTGDragonShield/original/Gold_Multicolor.csv
07): MTGManager/original/Mountains.csv vs MTGDragonShield/original/Mountains.csv
08): MTGManager/original/Swamps.csv vs MTGDragonShield/original/Swamps.csv
09): MTGManager/original/White_Collection.csv vs MTGDragonShield/original/White_Collection.csv
10): MTGManager/original/Blue_Collection.csv vs MTGDragonShield/original/Blue_Collection.csv
11): MTGManager/original/Red_Collection.csv vs MTGDragonShield/original/Red_Collection.csv
12): MTGManager/original/Forest

**Brilliant**!! It looks that the two list of `Collection` instances are aligned, so we could `zip` over the two lists, and get _matching_ collections that we can later compare.

## 1. Data format and layout


Without further ado, let's dig into the data of the same collections from the two App versions so to have a quick look at similarities and differences between the _old_ and _new_ data formats:

In [9]:
# Test data Loading
mm1_file = manager_files[1]
ds1_file = dshield_files[1]

mm1 = pd.read_csv(mm1_file)
ds1 = pd.read_csv(ds1_file)

In [10]:
mm1.head()

,Quantity,Name,Code,PurchasePrice,Foil,Condition,Language,PurchaseDate
0,2,Abandoned Outpost,ODY,0.0,0,0,0,26/1/2021
1,1,Abandoned Outpost,ODY,0.0,0,0,3,9/5/2020
2,1,Abandoned Outpost,ODY,0.0,0,0,6,9/5/2020
3,2,Adarkar Wastes,5ED,0.0,0,0,0,9/5/2020
4,2,Adarkar Wastes,6ED,0.0,0,0,0,9/5/2020


In [11]:
ds1.head()

,Quantity,Name,Expansion Code,Expansion Name,PurchasePrice,Foil,Condition,Language,PurchaseDate
0,2,Abandoned Outpost,ody,Odyssey,0.0,False,NearMint,English,20210126
1,1,Abandoned Outpost,ody,Odyssey,0.0,False,NearMint,French,20200509
2,1,Abandoned Outpost,ody,Odyssey,0.0,False,NearMint,Japanese,20200509
3,2,Adarkar Wastes,5ed,Fifth Edition,0.0,False,NearMint,English,20200509
4,2,Adarkar Wastes,6ed,Classic Sixth Edition,0.0,False,NearMint,English,20200509


<ins>**At a first glance...**</ins>

Of course, (**export**) data format has changed across the two generations of the App (and I imagine, the backend DB too). 

At a first glance, it looks like that both the backend DB has changed (e.g. see `Expansion Code` and `Expansion Name`), and the export feature from the App frontend has been improved  (e.g. <span class="codemono"> `Foil: [0, 1] => Foil: [False, True]` </span>). Besides, `PurchaseDate` field has also been changed into a more portable _format_, whereas `ExpansionCode` entries have been turned `lowercase`.

In order to align the two data formats, some pre-processing operations should be applied: 

* Column names should be aligned (e.g. `Code` and `Expansion Code`);
* `ExpansionCode` column should be turned all in lowercase - for comparability;
* `Foil` should be made boolean
* `PurchaseDate` should be processed as `datatime`
* `ExpansionCode` should be processd as `Categorical`
* `Condition` and `Language` should be made `Categorical`

**Notes** on `Categorical`:
- Vales ranges (i.e. Categories are recovedered from the App)
- `Conditions` (_from the app_): 
    - (Old): `Near Mint; Lightly Played; Moderately Played; Heavily Played; Damaged`
    - (New): `Mint; Near Mint; Excellent; Good; LightPlayed; Played; Poor`
- `Language` (_from the app_):
    - (Old): `English; German; Portuguese; French; Italian; Spanish; Japanese; Chinese S.; Russian; Chinese T.; Korean`
    - (New): `English; French; German; Italian; Japanese; Korean; Portuguese; Russian; Simplified Chinese; Spanish; Traditional Chinese`

### 1.1 Reading and Parsing data _layout_: 

So, after those initial consideration on data format, a more _accurate_ method to read in the data from the two formats should look like the `read_mtg_data` function reported below:

```python
def read_mtg_data(filepath: Path) -> pd.DataFrame:
    """This function will be used throughout the notebook as the reference function """
    df = pd.read_csv(filepath, header=0, parse_dates=["PurchaseDate", ], quotechar='"')
    # Code column has been renamed ExpansionCode in the NEW MTGManager format
    old_data_layout = "Code" in df.columns
    if len(df.columns) == 8:
        # re-map columns, mostly Code -> ExpansionCode
        df.columns = ["Quantity", "Name", "ExpansionCode", "PurchasePrice", 
                      "Foil", "Condition", "Language", "PurchaseDate"]
    else: # NEW MTGDragonShield Format
        df.columns = ["Quantity", "Name", "ExpansionCode", "ExpansionName", 
                      "PurchasePrice", "Foil", "Condition", "Language", "PurchaseDate"]
        
    if old_data_layout:
        # transform all expansion code in lower-case
        df["ExpansionCode"] = df.ExpansionCode.apply(lambda c: c.lower())
        # Re-map Condition to the new categorical scale
        df["Condition"] = df.Condition.map({0: "NearMint", 1: "Excellent", 2: "Good", 3: "Played", 4: "Poor"})
        # remap languages to match the new labels
        # English; German; Portuguese; French; Italian; Spanish; Japanese; Chinese S.; Russian; Chinese T.; Korean
        df["Language"] = df.Language.map({0: "English", 1: "German", 2: "Portuguese", 3: "French", 4: "Italian", 
                                          5: "Spanish", 6: "Japanese", 7: "Simplified Chinese", 8: "Russian", 
                                          9: "Traditional Chinese", 10: "Korean"})

    # Map Foil as Boolean
    df["Foil"] = df.Foil.map(bool)
    
    # Conditional: Ordinal Type
    # Poor < Played < LightPlayed < Good, < Excellent < Near Mint < Mint Mint
    df["Condition"] = pd.Categorical(df.Condition.values, ordered=True, 
                                     categories=["Poor", "Played", "LightPlayed", "Good", 
                                                 "Excellent", "NearMint", "Mint"])
    # Language: Nominal Type
    df["Language"] = pd.Categorical(df.Language.values)
    return df
```

In particular: 
1. Read the CSV making sure that `PurchaseDate` is correctly interpreted as `date` field;
2. Check if _current_ file is of `old_format` by looking for the `Code` column. If so:
    2.1 Transform all `Code` into lowercase (for comparability);
    2.2 Remap `Condition` codes to nominal values as in the _new_ format (later as `Ordinal Categorical`)
    2.3 Remap `Language` codes to nominal values (derived by order of appearance of languages in the old app)
3. Reset and reorder columns;
3. Remap `Foil` as Boolean series, `Condition` as categorical (_ordinal_), and `Language` as categorical (nominal)

#### 1.1.1 The `Collection` class

To better handle data processing (and most importantly **data encapsulation**), the special `Collection` class will be defined and used throughout this notebook. 

**Goal**:

The very aim of `Collection` is to encapsulate any _metadata_ about a single collection (e.g. name, reference app, total number of cards), as well as allow for an easy (proxy) access to cards contained in the collection. To do so, the above `read_mtg_data` method will be used to load datafiles, and to normalise their layout so that collections will be _as comparable, as possible_ (at least for data layout). 
**Besides** (relevant to our purposes), it will also provide an method to _compare_ two collections to determine their <ins>delta difference</ins>. This will be as easy as **subtracting** one collection from another (i.e. `C1 - C2`).

**NOTE**: the full code for the `Collection` class is available in the `mtg` Python module, along with its corresponding tests (see [`collection.py`](./mtg/collection.py))

In [12]:
from mtg import Collection

In [13]:
# Generate the two Apps list of Collections
mtg_manager_collection = [Collection(f, source="MTGManager") for f in manager_files]
mtg_dshield_collection = [Collection(f, source="MTGDragonShield") for f in dshield_files]

In [14]:
mm1 = mtg_manager_collection[1]
ds1 = mtg_dshield_collection[1]

In [15]:
mm1.name, ds1.name

('MTGManager/Non basic Lands', 'MTGDragonShield/Non basic Lands')

In [16]:
mm1.head()

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
0,2,Abandoned Outpost,ody,0.0,False,NearMint,English,2021-01-26
1,1,Abandoned Outpost,ody,0.0,False,NearMint,French,2020-09-05
2,1,Abandoned Outpost,ody,0.0,False,NearMint,Japanese,2020-09-05
3,2,Adarkar Wastes,5ed,0.0,False,NearMint,English,2020-09-05
4,2,Adarkar Wastes,6ed,0.0,False,NearMint,English,2020-09-05


In [17]:
ds1.head()

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
0,2,Abandoned Outpost,ody,Odyssey,0.0,False,NearMint,English,2021-01-26
1,1,Abandoned Outpost,ody,Odyssey,0.0,False,NearMint,French,2020-05-09
2,1,Abandoned Outpost,ody,Odyssey,0.0,False,NearMint,Japanese,2020-05-09
3,2,Adarkar Wastes,5ed,Fifth Edition,0.0,False,NearMint,English,2020-05-09
4,2,Adarkar Wastes,6ed,Classic Sixth Edition,0.0,False,NearMint,English,2020-05-09


As a matter of fact, we can access `pandas.DataFrame` methods proxied by a `Collection` exactly as we wanted. Moreover, the two collections from the two different data formats have been _aligned_ in the layout thanks to the specialised data loading function.

🔍
In the following sections, we will dig more into some data fields, namely `PurchaseDate`, `ExpansionCode` and `ExpansionName`.

### 1.2 `Quantity` check

First off, let's compare if _nominal_ quantities (i.e. those reported in the _old_ and _new_ App) do match for each collection

In [18]:
MTG_MANAGER = {
    "Red Collection":   9428,
    "White Collection": 4748,
    "Black Collection":  597,
    "Green Collection":  682,
    "Blue Collection":  5278,
    "Gold Multicolor":  1592,
    "Non basic Lands":  1580,
    "Islands":           231,
    "Plains":            261,
    "Swamps":            248,
    "Forests":           256,
    "Mountains":         416,
    "Artifacts":        3571
}

MTG_DRAGONSHIELD = {
    "Red Collection":   9425,
    "White Collection": 4748,
    "Black Collection":  597,
    "Green Collection":  682,
    "Blue Collection":  5278,
    "Gold Multicolor":  1592,
    "Non basic Lands":  1580,
    "Islands":           231,
    "Plains":            261,
    "Swamps":            248,
    "Forests":           255,
    "Mountains":         416,
    "Artifacts":        3571
}

In [19]:
for c_name in MTG_MANAGER:
    manager_count, dshield_count = MTG_MANAGER[c_name], MTG_DRAGONSHIELD[c_name]
    if manager_count != dshield_count:
        print(f"Non-matching Count for {c_name}: {manager_count} (MTG Manager) vs {dshield_count} (MTG DragonShield)")

Non-matching Count for Red Collection: 9428 (MTG Manager) vs 9425 (MTG DragonShield)
Non-matching Count for Forests: 256 (MTG Manager) vs 255 (MTG DragonShield)


⚠️ 

So apparently there are **3** cards missing in the `Red Collection` and **1** card missing in `Forests`.

Let's now check that these counts corresponds to the total **sum of** `Quantity` field reported in each collection, that is `len(Collection)`:

In [20]:
for collection in mtg_manager_collection:
    nominal_count = MTG_MANAGER[collection.label]
    mismatch = nominal_count != len(collection)
    if mismatch:
        print(f"⚠️ {collection.name}: {nominal_count} (Nominal, in App) vs {len(collection)} (Actual, in Data)")
    else:
        print(f"✅ {collection.name}")

✅ MTGManager/Black Collection
✅ MTGManager/Non basic Lands
✅ MTGManager/Artifacts
✅ MTGManager/Green Collection
✅ MTGManager/Islands
✅ MTGManager/Gold Multicolor
✅ MTGManager/Mountains
✅ MTGManager/Swamps
✅ MTGManager/White Collection
✅ MTGManager/Blue Collection
✅ MTGManager/Red Collection
✅ MTGManager/Forests
✅ MTGManager/Plains


In [21]:
for collection in mtg_dshield_collection:
    nominal_count = MTG_DRAGONSHIELD[collection.label]
    mismatch = nominal_count != len(collection)
    if mismatch:
        print(f"⚠️ {collection.name}: (Nominal, Actual) {nominal_count} vs {len(collection)}")
    else:
        print(f"✅ {collection.name}")

⚠️ MTGDragonShield/Black Collection: (Nominal, Actual) 597 vs 598
✅ MTGDragonShield/Non basic Lands
⚠️ MTGDragonShield/Artifacts: (Nominal, Actual) 3571 vs 3582
⚠️ MTGDragonShield/Green Collection: (Nominal, Actual) 682 vs 683
✅ MTGDragonShield/Islands
⚠️ MTGDragonShield/Gold Multicolor: (Nominal, Actual) 1592 vs 1594
✅ MTGDragonShield/Mountains
✅ MTGDragonShield/Swamps
⚠️ MTGDragonShield/White Collection: (Nominal, Actual) 4748 vs 4758
⚠️ MTGDragonShield/Blue Collection: (Nominal, Actual) 5278 vs 5291
⚠️ MTGDragonShield/Red Collection: (Nominal, Actual) 9425 vs 9457
✅ MTGDragonShield/Forests
✅ MTGDragonShield/Plains


⚠️

So, there is certainly something going on here in the data from the _new_ app: several mismatches have been found, and all counts are **all** _higher_ than nominal values.

🔍 This looks suspicious, and requires further investigations, as there is some data _duplicated_!

**Edit//Hint**: For possible explanations jump to Section [2.2.1](#Double-Cards-in-DragonShield-Data)

### 1.3 `PurchaseDate`

Since the format of data has changed across the two apps for dates, let's check for <ins>any inconsistency</ins> (still using the reference collections `mm1`, and `ds1` as examples). 

In particular, what I'd be interested in inspecting is whether some "troublesome" dates are handled properly. 

This is the case of dates like the one showed above☝️, i.e. `2020-05-09`. Is this `9th of May` or `5th of September` in the old data? 

In [22]:
reference_date = "2020-05-09"
ds1[((ds1.PurchaseDate==reference_date) & (ds1.Name=="Adarkar Wastes"))]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
3,2,Adarkar Wastes,5ed,Fifth Edition,0.0,False,NearMint,English,2020-05-09
4,2,Adarkar Wastes,6ed,Classic Sixth Edition,0.0,False,NearMint,English,2020-05-09
6,5,Adarkar Wastes,ice,Ice Age,0.0,False,NearMint,English,2020-05-09


In [23]:
mm1[((mm1.PurchaseDate==reference_date) & (mm1.Name=="Adarkar Wastes"))]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate


🤔 **mmm**, this looks suspicious..I am afraid I might be probably right... let's see: 

In [24]:
old_reference_date = "2020-09-05"
mm1[((mm1.PurchaseDate==old_reference_date) & (mm1.Name=="Adarkar Wastes"))]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
3,2,Adarkar Wastes,5ed,0.0,False,NearMint,English,2020-09-05
4,2,Adarkar Wastes,6ed,0.0,False,NearMint,English,2020-09-05
6,5,Adarkar Wastes,ice,0.0,False,NearMint,English,2020-09-05


⚠️ So apparently there is a mismatch in how dates have been exported//imported from the old to the new collections, respectively. Just to be sure that this is actually an error in the data, rather than a _reading_ error through pandas, let's have a look at the original CSV files:

In [25]:
!head -n 7 $mm1.filepath

Quantity,Name,Code,PurchasePrice,Foil,Condition,Language,PurchaseDate
2,"Abandoned Outpost",ODY,0.0,0,0,0,26/1/2021
1,"Abandoned Outpost",ODY,0.0,0,0,3,9/5/2020
1,"Abandoned Outpost",ODY,0.0,0,0,6,9/5/2020
2,"Adarkar Wastes",5ED,0.0,0,0,0,9/5/2020
2,"Adarkar Wastes",6ED,0.0,0,0,0,9/5/2020
1,"Adarkar Wastes",6ED,0.0,1,0,0,12/7/2020


In [26]:
!head -n 7 $ds1.filepath

Quantity,Name,Expansion Code,Expansion Name,PurchasePrice,Foil,Condition,Language,PurchaseDate
2,"Abandoned Outpost",ody,Odyssey,0.0,false,NearMint,English,20210126
1,"Abandoned Outpost",ody,Odyssey,0.0,false,NearMint,French,20200509
1,"Abandoned Outpost",ody,Odyssey,0.0,false,NearMint,Japanese,20200509
2,"Adarkar Wastes",5ed,Fifth Edition,0.0,false,NearMint,English,20200509
2,"Adarkar Wastes",6ed,Classic Sixth Edition,0.0,false,NearMint,English,20200509
1,"Adarkar Wastes",6ed,Classic Sixth Edition,0.0,true,NearMint,English,20200712


❌: There is an actual inconsistency in the data that cannot be solved by changing the format while reading data in (see first entry). I have reasons to believe that months and days have been consistenly swapped, but this would be tiresome and pointless to do. 

So let's just **document the issue** (to be reported) and let's get rid of dates when we will end up doing the actual comparison (as it is also quite useless). 

### 1.4 `ExpansionCode`

Now let's try to see whether there is a full `1-to-1` matching between `ExpansionCode` in the two data formats:

In [27]:
ds1.ExpansionCode.unique() == mm1.ExpansionCode.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True,  True, False])

As (kinda) _expected_, some **mismatches** exist! So we need to work around those to improve our ability to identify differences in the collections.

Let's have a look at these actual codes:

In [28]:
ds_codes = ds1.ExpansionCode.unique()
ds_codes

array(['ody', '5ed', '6ed', 'ice', 'hml', 'mrd', 'inv', 'tmp', 'ala',
       'csp', 'tsb', 'dis', 'ima', 'dgm', 'rtr', 'rna', 'mir', 'all',
       'ons', 'ori', '2ed', 'usg', 'dst', 'ktk', 'frf', 'rav', 'grn',
       'gtc', 'fem', 'dom', 'bfz', 'akh', 'eld', 'apc', 'soi', 'm19',
       '8ed', 'chr', 'arn', 'exo', 'isd', 'vis', 'ogw', 'e01', 'hou',
       'cma', 'mma', 'pls', 'ddq', 'm11', 'xln', 'wwk', 'm13', 'm12',
       'rix', 'con', 'ulg', 'a25', 'thb', '10e', 'tsp', 'wth', 'mm3',
       'c16', 'shm', 'gpt', 'leg', 'mmq', 'zen', '7ed', '9ed', 'ddh',
       'c13', 'jud', 'fut', 'v12', 'drk', 'ddf', '4ed', 'g05', 'uma',
       'emn', 'chk', 'c15', 'pcy', 'cn2', 'eve', 'cp3', 'kld', 'm15',
       'ddp', 'ath', 'atq', 'hop', 'm10', 'btd', 'dds', 'tor', 'ddm',
       'ths', 'scg', 'bok', 'nem', 'f04', 'plc', 'lrw', 'md1', 'pg07'],
      dtype=object)

In [29]:
mm_codes = mm1.ExpansionCode.unique()
mm_codes

array(['ody', '5ed', '6ed', 'ice', 'hml', 'mrd', 'inv', 'tmp', 'ala',
       'csp', 'tsb', 'dis', 'ima', 'dgm', 'rtr', 'rna', 'mir', 'all',
       'ons', 'ori', '2ed', 'usg', 'dst', 'ktk', 'frf', 'rav', 'grn',
       'gtc', 'fem', 'dom', 'bfz', 'akh', 'eld', 'apc', 'soi', 'm19',
       '8ed', 'chr', 'arn', 'exo', 'isd', 'vis', 'ogw', 'e01', 'hou',
       'cma', 'mma', 'pls', 'ddq', 'm11', 'xln', 'wwk', 'm13', 'm12',
       'rix', 'con', 'ulg', 'a25', 'thb', '10e', 'tsp', 'wth', 'mm3',
       'c16', 'shm', 'gpt', 'leg', 'mmq', 'zen', '7ed', '9ed', 'ddh',
       'c13', 'jud', 'fut', 'v12', 'drk', 'ddf', '4ed', 'pjgp', 'uma',
       'emn', 'chk', 'c15', 'pcy', 'cn2', 'eve', 'cp3', 'kld', 'm15',
       'ddp', 'ath', 'atq', 'hop', 'm10', 'btd', 'dds', 'tor', 'ddm',
       'ths', 'scg', 'bok', 'nms', 'pfnm', 'plc', 'lrw', 'md1', 'pgtw'],
      dtype=object)

In [30]:
import numpy as np 

non_matching = np.where(mm_codes != ds_codes)[0]

mm_codes[non_matching], ds_codes[non_matching]

(array(['pjgp', 'nms', 'pfnm', 'pgtw'], dtype=object),
 array(['g05', 'nem', 'f04', 'pg07'], dtype=object))

In [31]:
ds1[ds1.ExpansionCode.isin(['g05', 'f04', 'pg07'])]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
301,1,Mishra's Factory,g05,Judge Gift Cards 2005,0.0,False,NearMint,English,2020-05-10
539,1,Treetop Village,f04,Friday Night Magic 2004,0.0,True,NearMint,English,2020-05-09
622,1,Zoetic Cavern,pg07,Gateway 2007,0.0,True,NearMint,English,2020-07-19


In [32]:
mm1[mm1.ExpansionCode.isin(['pjgp', 'pfnm', 'pgtw'])]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
301,1,Mishra's Factory,pjgp,0.0,False,NearMint,English,2020-10-05
539,1,Treetop Village,pfnm,0.0,True,NearMint,English,2020-09-05
622,1,Zoetic Cavern,pgtw,0.0,True,NearMint,English,2020-07-19


#### 1.4.1 Non-matching `ExpansionCode` Codes in the old data format

It looks that these mismatches happen with several expansions. So, let's now collect all the `ExpansionCode ==> ExpansionName` mapping in the _new_ data format.

This will be serve two purposes: 

- (a) collect all the (unique) expansion codes available throught the list of collections; and 
- (b) also collect their corresponding _Expansion Name_. 

In [33]:
from typing import Sequence, Dict

def gather_expansions_map(collections: Sequence[Collection]) -> Dict[str, str]:
    """This function will generate the corresponding expansion name for each expansion code found
    (in the new data format data)
    """
    codes_names_map = dict()
    for c in collections:
        codes_names = c[["ExpansionCode", "ExpansionName"]].drop_duplicates().values
        cmap = {k:v for k, v in zip(codes_names[:, 0], codes_names[:, 1])}
        print(f"Found {len(cmap)} substitutions from {c.label}")
        codes_names_map.update(cmap)
    return codes_names_map
    

In [34]:
# NOTE: Only the new DragonShield collection will be considered for this (of course)
EXPANSION_CODENAMES_MAP = gather_expansions_map(mtg_dshield_collection)

Found 85 substitutions from Black Collection
Found 108 substitutions from Non basic Lands
Found 124 substitutions from Artifacts
Found 99 substitutions from Green Collection
Found 30 substitutions from Islands
Found 87 substitutions from Gold Multicolor
Found 50 substitutions from Mountains
Found 33 substitutions from Swamps
Found 137 substitutions from White Collection
Found 148 substitutions from Blue Collection
Found 178 substitutions from Red Collection
Found 35 substitutions from Forests
Found 35 substitutions from Plains


In [35]:
len(EXPANSION_CODENAMES_MAP)

228

Having the `EXPANSION_CODENAMES_MAP`, we can now look for all the `ExpansionCode` in the **Old Collection Data Format** not having a corresponding representative in this dictionary map. 

Those expansions code in the **old** format will be the candidate to be _replaced_, accordingly.

In [36]:
from collections import defaultdict

# We will be saving for each Non-found code, the corresponding Collection where the code has been found
codes_not_found = defaultdict(list)
EXP_CODES = set(EXPANSION_CODENAMES_MAP.keys())

for mm in mtg_manager_collection:
    mm_codes = mm.ExpansionCode.unique()
    non_matching = filter(lambda c: c not in EXP_CODES, mm_codes)
    for code in non_matching:
        codes_not_found[code].append(mm.label)

In [37]:
print("Non-matching Codes: ", len(codes_not_found))
display(codes_not_found)

Non-matching Codes:  14


defaultdict(list,
            {'dd3_dvd': ['Black Collection'],
             'po2': ['Black Collection',
              'Green Collection',
              'Islands',
              'Mountains',
              'Swamps',
              'White Collection',
              'Red Collection',
              'Forests',
              'Plains'],
             'nms': ['Black Collection',
              'Non basic Lands',
              'Artifacts',
              'Green Collection',
              'White Collection',
              'Blue Collection',
              'Red Collection'],
             'pjgp': ['Non basic Lands', 'Red Collection'],
             'pfnm': ['Non basic Lands',
              'Artifacts',
              'Gold Multicolor',
              'White Collection',
              'Blue Collection',
              'Red Collection'],
             'dd3_gvl': ['Green Collection'],
             'pwpn': ['Gold Multicolor', 'Blue Collection', 'Red Collection'],
             'dd3_jvc': ['Blue Collection'],
   

**Fortunately**, _non-matching_ expansion codes are not that many (`14` in total), and some of them are "`EASYFIX`" (e.g. `dd3_dvd --> dvd`).

Now all we need to do is to look for others _less intuitive_ substitutions, comparing entries in the old collection with the "matching" ones in the new format.

Let's also count which collection has the higest number of those missing expansion codes:

In [38]:
from collections import Counter

coll_counter = Counter([coll for code in codes_not_found for coll in codes_not_found[code]])
coll_counter.most_common()

[('Red Collection', 10),
 ('Blue Collection', 6),
 ('Black Collection', 3),
 ('Green Collection', 3),
 ('White Collection', 3),
 ('Non basic Lands', 3),
 ('Artifacts', 2),
 ('Gold Multicolor', 2),
 ('Islands', 1),
 ('Mountains', 1),
 ('Swamps', 1),
 ('Forests', 1),
 ('Plains', 1)]

**So**:
- `RedCollection` (as expected) has the major number of hits, which will also cover replacements for all interesting cases (but `plpa`);
- `BlueCollection` will be used for `plpa`;
- Remaining expansions are straightforward and will be re-mapped manually (e.g. `dd3_gvl:gvl`, `dd3_jvc:jvc`).

#### 1.4.2 Look for Potential Replacement Candidates:

Now the plan is two merge the data from the two collections, and look at potential candidates for `target` `ExpansionCode`:

In [39]:
# See previous collections' lists for reference to these indices
blue_coll_idx = 9
red_coll_idx = 10

def merge_collections(idx: int) -> pd.DataFrame:
    """merge the two dataframes from collections at a given index looking at potential expansion code replace candidates"""
    old_c = mtg_manager_collection[idx]
    new_c = mtg_dshield_collection[idx]
    return pd.merge(old_c.data, new_c.data, 
                    on=["Name", "Quantity", "Condition", "Language"], 
                    how="inner", suffixes=("_OLD", "_NEW"))

In [40]:
red_df = merge_collections(red_coll_idx)
blue_df = merge_collections(blue_coll_idx)

In [41]:
for code in codes_not_found:
    if code.startswith("dd3"):
        continue  # skip - no need to fuss
    
    print("Looking for code: ", code)
    if code == "plpa": 
        df = blue_df
    else: 
        df = red_df
    sel = df[df.ExpansionCode_OLD == code][["Name", "ExpansionCode_OLD", 
                                            "ExpansionCode_NEW", "ExpansionName"]].drop_duplicates()
    display(sel.head())

Looking for code:  po2


,Name,ExpansionCode_OLD,ExpansionCode_NEW,ExpansionName
326,Brimstone Dragon,po2,p02,Portal Second Age
517,Cunning Giant,po2,p02,Portal Second Age
1143,Goblin Cavaliers,po2,p02,Portal Second Age
1180,Goblin Firestarter,po2,p02,Portal Second Age
1242,Goblin Mountaineer,po2,p02,Portal Second Age


Looking for code:  nms


,Name,ExpansionCode_OLD,ExpansionCode_NEW,ExpansionName
82,Ancient Hydra,nms,nem,Nemesis
100,Arc Mage,nms,nem,Nemesis
297,Bola Warrior,nms,nem,Nemesis
602,Downhill Charge,nms,nem,Nemesis
933,Flame Rift,nms,nem,Nemesis


Looking for code:  pjgp


...

,Name,ExpansionCode_OLD,ExpansionCode_NEW,ExpansionName
838,Fireball,pmpr,p05,Magic Player Rewards 2005
839,Fireball,pmpr,pd2,Premium Deck Series: Fire and Lightning
840,Fireball,pmpr,pmei,Magazine Inserts
841,Fireball,pmpr,3ed,Revised Edition
842,Fireball,pmpr,2ed,Unlimited Edition


Looking for code:  rqs


,Name,ExpansionCode_OLD,ExpansionCode_NEW,ExpansionName
858,Fireball,rqs,p05,Magic Player Rewards 2005
859,Fireball,rqs,pd2,Premium Deck Series: Fire and Lightning
860,Fireball,rqs,pmei,Magazine Inserts
861,Fireball,rqs,3ed,Revised Edition
862,Fireball,rqs,2ed,Unlimited Edition


Looking for code:  parl


,Name,ExpansionCode_OLD,ExpansionCode_NEW,ExpansionName
1088,Genju of the Spires,parl,a25,Masters 25
1089,Genju of the Spires,parl,pal05,Arena League 2005
1117,Glacial Ray,parl,pal04,Arena League 2004


**Conclusions**: 

Some expansions could be fixed automatically (e.g. `nms` to `nem`; `po2` to `p02`), **but** some others (e.g. `fnm`) 
require *extra* work. 

At this point, we need to resort to having a **ScryFall** database oracle available to fix this! 💪

**MANUAL MAPPING**

In [42]:
list(codes_not_found.keys())

['dd3_dvd',
 'po2',
 'nms',
 'pjgp',
 'pfnm',
 'dd3_gvl',
 'pwpn',
 'dd3_jvc',
 'plpa',
 'pthb',
 'dd3_evg',
 'pmpr',
 'rqs',
 'parl']

In [43]:
EXP_CODES_REPLACE_MAP = {
    "dd3_dvd" : "dvd",
    "dd3_gvl": "gvl",
    "dd3_jvc": "jvc",
    "dd3_evg": "evg",
    "po2": "p02",
    "pthb": "vthb",
    "nms": "nem",
    "rqs": "p05",  # only Fireball
    # special PROMO sets requiring Scryfall
    "pjgp": "g",
    "pfnm": "f",
    "pwpn": "pwp",
    "plpa": "p",
    "pmpr": "p",
#     "parl": "pal"
}

PROMO_EXPANSIONS = ["pjgp", "pfnm", "pwpn", "plpa", "pmpr"]

### 1.5 Summary and Wrap up

This concludes our inspection on data formats, and comparison between _old_ and _new_ data layout. 

After this preliminary analysis, we can conclude that:

- (A) We spotted an inconsistency in the two data formats for what concerns _PurchaseDate_: On this note, there is much we can do. (**To be reported** to DragonShield Support).
- (B) We also verified the presence of inconsistent `ExpansionCode` across the two formats: some of those can be easily fixed, but _promo_ expansions require extra (_external_) information to be fixed. For this reason, a `ScryfallDB` oracle will be set up and used.

---

## 2. `ScryfallDB` Oracle

Now we are going to open our **next**, and potentially _last_, chapter in this exploratory analysis of MTG-Manager Collection data: the **Scryfall** Card Oracle Database. 

We will be using the **Oracle** to:
- Get all the Cards (limited to English _language_), collecting their corresponding `Set Code`, `Set Name`, and `Set Type`
- Get the list of all `ExpansionCode` and their corresponding `Expansion Name`:
    - In this way, we can also expand and complete the old data format with the corresponding `ExpansionName` for a more thorough comparison!

**Furthermore**, we can also look for any _non-existing_ cards in the `Collection`, in order to sanitise any data inconsistency!

**NOTE**: The `ScryfallDB` class is available in the `mtg` Python Package included in the repository (see [`scryfall.py`](./mtg/scryfall.py)

In [44]:
from mtg import ScryfallDB

# The Scryfall-Default database will be used.
# If the DB file does not exist, it will be automatically 
# downloaded first from Scryfall CDN.
CARDS_ORACLE = ScryfallDB()

Loading Full Database file
Cards: 314861 [00:04, 68491.84/s] 

Loading Cards from Database into Oracle


In [45]:
print(f"Total number of cards in the Oracle: {len(CARDS_ORACLE)}")

Total number of cards in the Oracle: 53578


**Comparison with `Scryfall` Oracle**

In this section we will check all the data collected from both the _old_ and the _new_ data collections. 

In particular, we will:

1. Look for any `ExpansionCode` not found in the Oracle, and map corresponding `ExpansionName`; 

2. Look for any non-existing card name due to inconsistency in the DB that needs fixing;

3. Re-map all `ExpansionCode` from the _old_ data format to codes in the Oracle;
    1. FIX all `promo` expansions with specifics of the single card!

4. Add the `ExpansionName` column to the _old format_ data

5. Save the new CSV (fixed) data files.

In [46]:
from itertools import chain
# Group together all the available Collection instances
collections = list(chain(mtg_manager_collection, mtg_dshield_collection))
len(collections)

26

### 2.1 `ExpansionCode` integrity

We will be using only codes included in the **NEW** data format as those are used as reference for comparison. Moreover, the _new_ data layout is the only one to include for each code its corresponding `ExpansionName`.

In [47]:
# So, let's then take the first half
dshield_collections = collections[13:]
expansion_codenames = set()
for i, c in enumerate(dshield_collections):
    print(f"\t {i+1:02d}) {c.source}-{c.label}")
    code_names = c[["ExpansionCode", "ExpansionName"]].drop_duplicates().values
    code_names = set([(code, name) for code, name in zip(code_names[:, 0], code_names[:, 1])])
    expansion_codenames.update(code_names)
    
print(f"Collected {len(expansion_codenames)} Code-Names Pairs")

	 01) MTGDragonShield-Black Collection
	 02) MTGDragonShield-Non basic Lands
	 03) MTGDragonShield-Artifacts
	 04) MTGDragonShield-Green Collection
	 05) MTGDragonShield-Islands
	 06) MTGDragonShield-Gold Multicolor
	 07) MTGDragonShield-Mountains
	 08) MTGDragonShield-Swamps
	 09) MTGDragonShield-White Collection
	 10) MTGDragonShield-Blue Collection
	 11) MTGDragonShield-Red Collection
	 12) MTGDragonShield-Forests
	 13) MTGDragonShield-Plains
Collected 228 Code-Names Pairs


Let's check how many _Expansion_ (`Code:Name`) are present in the oracle 

In [48]:
print("(Code:Name) Pairs in Scyfall Oracle: ", len(CARDS_ORACLE.expansion_codename_map))

(Code:Name) Pairs in Scyfall Oracle:  638


In [49]:
for code, name in expansion_codenames:
    if not code in CARDS_ORACLE.expansion_codename_map:
        print(f"Expansion Code {code} NOT found in Oracle. ")
        print(f"Adding ({code}, {name}) to the Oracle...", end="")
        CARDS_ORACLE.add_expansion_code((code, name))
        print("Done!", end="\n\n")

Expansion Code veld NOT found in Oracle. 
Adding (veld, Throne of Eldraine Variants) to the Oracle...Done!

Expansion Code ppre NOT found in Oracle. 
Adding (ppre, Prerelease Events) to the Oracle...Done!

Expansion Code gk2_simic NOT found in Oracle. 
Adding (gk2_simic, Guild Kit: Simic) to the Oracle...Done!

Expansion Code gk2_orzhov NOT found in Oracle. 
Adding (gk2_orzhov, Guild Kit: Orzhov) to the Oracle...Done!

Expansion Code vthb NOT found in Oracle. 
Adding (vthb, Theros Beyond Death Variants) to the Oracle...Done!

Expansion Code gk1_izzet NOT found in Oracle. 
Adding (gk1_izzet, Guild Kit: Izzet) to the Oracle...Done!

Expansion Code gk1_dimir NOT found in Oracle. 
Adding (gk1_dimir, Guild Kit: Dimir) to the Oracle...Done!

Expansion Code gk1_boros NOT found in Oracle. 
Adding (gk1_boros, Guild Kit: Boros) to the Oracle...Done!

Expansion Code gk2_gruul NOT found in Oracle. 
Adding (gk2_gruul, Guild Kit: Gruul) to the Oracle...Done!

Expansion Code gk2_rakdos NOT found in O

In [50]:
print("Updated (Code:Name) Pairs in Scryfall Oracle: ", len(CARDS_ORACLE.expansion_codename_map))

Updated (Code:Name) Pairs in Scryfall Oracle:  648


⚠️ <br />
**So**, `10` Expansion Code (out of `228`) where **not found** in Oracle, and so were added. Those where:

```
(gk1_izzet, Guild Kit: Izzet)
(gk1_boros, Guild Kit: Boros)
(gk1_dimir, Guild Kit: Dimir)
(gk2_simic, Guild Kit: Simic)
(gk2_gruul, Guild Kit: Gruul)
(gk2_orzhov, Guild Kit: Orzhov)
(gk2_rakdos, Guild Kit: Rakdos)
(vthb, Theros Beyond Death Variants)
(ppre, Prerelease Events)
(veld, Throne of Eldraine Variants)
```

We do now have an Expansion Name for all the `ExpansionCode` we will encounter in cards collection!

### 2.2 Cards Integrity

The last _sanitisation_ operation regards actual `Card` names in the data, compared to the Oracle. 

<ins>In this section, we will be looking for cards not appearing in the `CARDS_ORACLE` that will require manual fixings in the data files.</ins>

#### Collect Missing Names

**First off**, let's collect all `Name` of cards not appearing in the `Scryfall` database for further examinations:

In [51]:
from typing import Tuple, Optional

def collect_missing_card_names(coll: Collection, oracle: ScryfallDB) -> Optional[Tuple[str]]:
    """Collect all names of cards not found in the input Oracle."""
    in_oracle = coll.Name.apply(lambda n: n in oracle)
    not_found = coll[~in_oracle]
    if len(not_found):
        return tuple(not_found.Name.unique())
    return None
    

In [52]:
missing_cards = partial(collect_missing_card_names, oracle=CARDS_ORACLE)
collections_with_missing_cards = list(filter(lambda p: p[1] is not None, 
                                             zip((c.name for c in collections), 
                                                 map(missing_cards, collections))))
print(f"{len(collections_with_missing_cards)} have cards not found in Scryfall DB")

8 have cards not found in Scryfall DB


In [53]:
for coll_name, missing_cards in collections_with_missing_cards:
    print(f"🔍 Missing Cards found in {coll_name}: {len(missing_cards)}")
    for c in missing_cards:
        print(f"\t {c}")

🔍 Missing Cards found in MTGManager/Black Collection: 1
	 Chosen of Markov
🔍 Missing Cards found in MTGManager/Artifacts: 5
	 Chalice of Life
	 Conqueror's Galleon
	 Cryptolith Fragment
	 Neglected Heirloom
	 Thraben Gargoyle
🔍 Missing Cards found in MTGManager/Green Collection: 2
	 Budoka Pupil
	 Moonscarred Werewolf
🔍 Missing Cards found in MTGManager/Gold Multicolor: 25
	 Alive
	 Arlinn Kord
	 Authority
	 Consign
	 Crime
	 Despair
	 Destined
	 Down
	 Farm
	 Finish
	 Fire
	 Give
	 Mind
	 Onward
	 Path of Mettle
	 Prepare
	 Protect
	 Ready
	 Reduce
	 Riches
	 Start
	 Struggle
	 Supply
	 Toil
	 Turn
🔍 Missing Cards found in MTGManager/Mountains: 4
	 mountain1
	 mountain2
	 mountain3
	 mountain4
🔍 Missing Cards found in MTGManager/White Collection: 7
	 Bushi Tenderfoot
	 Extricator of Sin
	 Loyal Cathar
	 Pious Evangel
	 circle of protection black
	 circle of protection green
	 circle of protection white
🔍 Missing Cards found in MTGManager/Blue Collection: 7
	 Curious Homunculus
	 Docen

⚠️

Going through the list of all the _missing_ cards found throughout the difference collections, we immediately notice that all the inconsistency found can be categories in two groups:

1. **Double cards** (those containing `//` in their name): In the _old_ version of the App, those cards were handled poorly, so that only _half_ of their name appears[+](#fnnote). 

2. Some cards include a number in their name (e.g. from `Homelands` or `Alliances`) matching their corresponding art. This makes cards not recognised in the DB. Besides, this behaviour seems to have changed in the _new_ app (**there is no MISSING cards in _new_ App collections**), so these numbers should be removed. 

<span id="fnnote"> **+**: A special option has been included in the `lookup` method of `ScryfallDB` to specifically filter search results with **Double Cards**.</span>

##### Double Cards in DragonShield Data

**Before** moving on, one last check: _How Double cards are handled in the New DragonShield App_?

Let's just take the very first _missing card_ found from `Black Collection`: `Chosen of Markov`

In [54]:
new_black_coll = mtg_dshield_collection[0]
new_black_coll[new_black_coll.Name.str.startswith("Chosen of Markov")]

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
42,1,Chosen of Markov // Markov's Servant,dka,Dark Ascension,0.05,False,NearMint,English,2018-07-29
178,1,Chosen of Markov // Markov's Servant,dka,Dark Ascension,0.05,False,NearMint,English,2018-07-29


In [55]:
old_black_coll = mtg_manager_collection[0]
old_black_coll[old_black_coll.Name.str.startswith("Chosen of Markov")]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
42,1,Chosen of Markov,dka,0.05,False,NearMint,English,2018-07-29


👀 🤔

This looks suspicious: it seems that the _new_ data format contains a **double entry** for **double cards**! (Makes sense, right?! 😅)

Let's see if this is just the case of `Black Collection` or not. Let's repeat the same with `Artifacts` where we found 5 missing cards in the _old_ data:

In [56]:
artifact_doubles = collections_with_missing_cards[1][1]  # retain just card names
new_artifact_coll = mtg_dshield_collection[2]
old_artifact_coll = mtg_manager_collection[2]
old_artifact_coll.name, new_artifact_coll.name

('MTGManager/Artifacts', 'MTGDragonShield/Artifacts')

In [57]:
for name in artifact_doubles:
    new_count = new_artifact_coll[new_artifact_coll.Name.str.startswith(name)].Quantity.sum()
    old_count = old_artifact_coll[old_artifact_coll.Name.str.startswith(name)].Quantity.sum()
    print(f"Card (Prefix) {name}: [NEW] vs [OLD]")
    print(f"--> {new_count} vs {old_count}")

Card (Prefix) Chalice of Life: [NEW] vs [OLD]
--> 4 vs 2
Card (Prefix) Conqueror's Galleon: [NEW] vs [OLD]
--> 2 vs 1
Card (Prefix) Cryptolith Fragment: [NEW] vs [OLD]
--> 4 vs 2
Card (Prefix) Neglected Heirloom: [NEW] vs [OLD]
--> 8 vs 4
Card (Prefix) Thraben Gargoyle: [NEW] vs [OLD]
--> 4 vs 2


⚠️   **That's it**

In the new data format, double cards correspond to <ins>double</ins> entries. Repeated entries _must_ be removed, otherwise those will be identified as missing records (not necessarily the case).

**Also**, this could _explain_ **number mismatch** (from nominal reference values) identified in Section [1.2](#1.2-Quantity-check)

---

#### (A) Fix Card Names 

Check for all names **not** found in `Scryfall` Oracle and replace them accordingly. In particular, the following operations will be applied: 
- remove any number present at the end of card's names (e.g. `mountain2`, `plains3`)
- look for all names missing in `Scryfall` oracle. This will correspond most likely to Double Cards, therefore:
    - look for potential (double card) replacement. If a single unique replacement will be found, **apply the change**.
    - otherwise, show list of candidates (if any).

In [58]:
import os
import re
from string import digits
from typing import Tuple

def fix_missing_card_names(coll: Collection, oracle: ScryfallDB, verbose: bool = True) -> None:
    """Check all the cards in the input Collection and: 
    - remove any number present at the end of names (e.g. mountain2, plains3)
    - look for any name that's missing in the oracle. This will be probably a double.
        - if there is one unique subsitution, apply replacement, otherwise show alternatives.
    """
    if verbose:
        print(f"[LOG]: Inspecting {coll.name}")
    
    # first resolve the issue with numbers in card name, if any!
    card_names = coll.Name
    stripped_names = card_names.apply(lambda n: n.rstrip(digits))
    coll.data["Name"] = stripped_names
    if verbose:
        prev_names = card_names.values
        strp_names = stripped_names.values
        diff = (strp_names != prev_names)
        if np.any(diff):
            print(f"[LOG] Names Replaced: \n {prev_names[diff]} \n {strp_names[diff]}")
    
    # Check cards that are not found in the oracle
    in_oracle = coll.Name.apply(lambda n: n in oracle)
    not_found = coll[~in_oracle]
    if not len(not_found):
        if verbose:
            print(f"[LOG] (No Change) All cards found in Oracle for {coll.name} ✅")
            print("="*20, end="\n\n")
        return
    
    # Looking for Double Cards to FIX (will cover 98% of the missing ones)
    yet_to_fix = {}
    card_names = not_found.Name.values.tolist()
    for i, name in enumerate(card_names):
        matches = tuple(oracle.lookup(name, expand_search=True, unique=True, doubles_only=True))
        if not matches or len(matches) > 1:
            yet_to_fix[name] = matches
        else:
            replace_name = matches[0].name
            coll[coll.Name == name, "Name"] = replace_name
            if verbose:
                print(f"[LOG] \t {name} --> {replace_name}")
    
    if verbose:
        all_done = " ✅" if not len(yet_to_fix) else " 👀 🔍"
        print(f"\n[LOG]: (Updated) {coll.name} {all_done}")

    if yet_to_fix:
        print("\nThe following cards resulted in multiple matchings and require manual checking: ")
        for i, name in enumerate(yet_to_fix):
            potential_matches = yet_to_fix[name]
            if not potential_matches:
                print(f"{i+1:02d}) {name} REQUIRES manual inspection due to some errors in its name.", end="\n\n")
            else:
                print(f"{i+1:02d}) {name} has {len(potential_matches)} candidate matches:")
                for j, match in enumerate(potential_matches): 
                    print(f"\t {j+1:02d}) {match.name}")
        print("", end="\n")
    print("="*20, end="\n\n")

In [59]:
from functools import partial
fix_card_names = partial(fix_missing_card_names, oracle=CARDS_ORACLE)

_ = list(map(fix_card_names, collections))

[LOG]: Inspecting MTGManager/Black Collection
[LOG] 	 Chosen of Markov --> Chosen of Markov // Markov's Servant

[LOG]: (Updated) MTGManager/Black Collection  ✅

[LOG]: Inspecting MTGManager/Non basic Lands
[LOG] (No Change) All cards found in Oracle for MTGManager/Non basic Lands ✅

[LOG]: Inspecting MTGManager/Artifacts
[LOG] 	 Chalice of Life --> Chalice of Life // Chalice of Death
[LOG] 	 Conqueror's Galleon --> Conqueror's Galleon // Conqueror's Foothold
[LOG] 	 Cryptolith Fragment --> Cryptolith Fragment // Aurora of Emrakul
[LOG] 	 Cryptolith Fragment --> Cryptolith Fragment // Aurora of Emrakul
[LOG] 	 Neglected Heirloom --> Neglected Heirloom // Ashmouth Blade
[LOG] 	 Thraben Gargoyle --> Thraben Gargoyle // Stonewing Antagonizer

[LOG]: (Updated) MTGManager/Artifacts  ✅

[LOG]: Inspecting MTGManager/Green Collection
[LOG] 	 Budoka Pupil --> Budoka Pupil // Ichiga, Who Topples Oaks
[LOG] 	 Moonscarred Werewolf --> Scorned Villager // Moonscarred Werewolf

[LOG]: (Updated) MTGM

#### (B) Dropping duplicates 

Remove all the double entries, for "double cards" (_only applies to the new data format_)

In [60]:
# This only happens for new data format. Otherwise, there should not be any duplicate
from itertools import chain
from typing import Optional

def drop_doublecards_duplicates(c: Collection, verbose: bool = True) -> None:
    db_cards = c[c.Name.str.contains("//")].apply(tuple, 1)
    double_cards = defaultdict(list)
    for index, entry in zip(db_cards.index, db_cards.values.tolist()):
        double_cards[entry].append(index)
    to_remove = list(chain.from_iterable(map(lambda l: sorted(l)[1:], 
                                             filter(lambda indices: len(indices) > 1, 
                                                    double_cards.values()))))
    if to_remove:
        if verbose:
            print(f"🔍 [LOG] Removing from {c.name} {len(to_remove)} duplicate{'s' if len(to_remove) > 1 else ''}:")
            display(c.loc[to_remove].Name.values.tolist())
            print("="*20, end="\n\n")
        c.drop(labels=to_remove, axis=0, inplace=True)

In [61]:
_ = list(map(drop_doublecards_duplicates, collections))

🔍 [LOG] Removing from MTGDragonShield/Black Collection 1 duplicate:


["Chosen of Markov // Markov's Servant"]


🔍 [LOG] Removing from MTGDragonShield/Artifacts 6 duplicates:


['Neglected Heirloom // Ashmouth Blade',
 'Cryptolith Fragment // Aurora of Emrakul',
 'Cryptolith Fragment // Aurora of Emrakul',
 'Chalice of Life // Chalice of Death',
 "Conqueror's Galleon // Conqueror's Foothold",
 'Thraben Gargoyle // Stonewing Antagonizer']


🔍 [LOG] Removing from MTGDragonShield/Green Collection 1 duplicate:


...

['Extricator of Sin // Extricator of Flesh',
 'Extricator of Sin // Extricator of Flesh',
 'Loyal Cathar // Unhallowed Cathar',
 'Pious Evangel // Wayward Disciple']


🔍 [LOG] Removing from MTGDragonShield/Blue Collection 8 duplicates:


['Curious Homunculus // Voracious Reader',
 'Docent of Perfection // Final Iteration',
 'Soul Seizer // Ghastly Haunting',
 'Grizzled Angler // Grisly Anglerfish',
 'Grizzled Angler // Grisly Anglerfish',
 "Ludevic's Test Subject // Ludevic's Abomination",
 "Ludevic's Test Subject // Ludevic's Abomination",
 'Startled Awake // Persistent Nightmare']


🔍 [LOG] Removing from MTGDragonShield/Red Collection 11 duplicates:


['Convicted Killer // Branded Howler',
 'Conduit of Storms // Conduit of Emrakul',
 'Vildin-Pack Outcast // Dronepack Kindred',
 'Vildin-Pack Outcast // Dronepack Kindred',
 'Smoldering Werewolf // Erupting Dreadwolf',
 'Gatstaf Arsonists // Gatstaf Ravagers',
 'Hinterland Hermit // Hinterland Scourge',
 'Instigator Gang // Wildblood Pack',
 'Reckless Waif // Merciless Predator',
 "Mondronen Shaman // Tovolar's Magehunter",
 'Village Messenger // Moonrise Intruder']

Let's now **re-check** collections' count with nominal values in App for **DragonShield**

In [62]:
for collection in mtg_dshield_collection:
    nominal_count = MTG_DRAGONSHIELD[collection.label]
    mismatch = nominal_count != len(collection)
    if mismatch:
        print(f"⚠️ {collection.name}: (Nominal, Actual) {nominal_count} vs {len(collection)}")
    else:
        print(f"✅ {collection.name}")

✅ MTGDragonShield/Black Collection
✅ MTGDragonShield/Non basic Lands
✅ MTGDragonShield/Artifacts
✅ MTGDragonShield/Green Collection
✅ MTGDragonShield/Islands
✅ MTGDragonShield/Gold Multicolor
✅ MTGDragonShield/Mountains
✅ MTGDragonShield/Swamps
✅ MTGDragonShield/White Collection
✅ MTGDragonShield/Blue Collection
✅ MTGDragonShield/Red Collection
✅ MTGDragonShield/Forests
✅ MTGDragonShield/Plains


🎉 🎉 🎉 🎉 🎉 

**ALL FIXED**

#### (C) Apply Remaining Fixes

**Brilliant** 🎉 

After the first round of **automatic** FIX to the file, only <ins>three</ins> Collections (in _old_ App set) require manual interventions, and honestly they do all makes sense as the proposed alternatives for doubles are all reasonable, and other fixes are due to **BUG** in the actual App.

**Note** ⚙️: There _could be_ indeed a way to automate this further, that is considering the colour of cards (although CMC is not included in the data) and the corresponding `ExpansionCode`, but I have the impression that this would require much more work than the manual fixing. So happy to proceed manually this time! 😬

**FIX 1**: _Gold-MultiColour_

In [63]:
gold_multicolours = collections[5]
gold_multicolours.name

'MTGManager/Gold Multicolor'

In [64]:
# Despair ==> Driven // Despair
gold_multicolours[(gold_multicolours.Name=="Despair"), "Name"] = "Driven // Despair"
# Down ==> Down // Dirty
gold_multicolours[(gold_multicolours.Name=="Down"), "Name"] = "Down // Dirty"
# Fire ==> Fire // Ice
gold_multicolours[(gold_multicolours.Name=="Fire"), "Name"] = "Fire // Ice"
# Mind ==> Spring // Mind
gold_multicolours[(gold_multicolours.Name=="Mind"), "Name"] = "Spring // Mind"
# Start ==> Start // Finish
gold_multicolours[(gold_multicolours.Name=="Start"), "Name"] = "Start // Finish"

**FIX 2:** _White_ Collection

In [65]:
white_coll = collections[8]
white_coll.name

'MTGManager/White Collection'

In [66]:
# White FIX:
# circle of protection black ==> Circle of Protection: Black
white_coll[white_coll.Name=="circle of protection black", "Name"] = "Circle of Protection: Black"

# circle of protection green ==> Circle of Protection: Green
white_coll[white_coll.Name=="circle of protection green", "Name"] = "Circle of Protection: Green"

# circle of protection white ==> Circle of Protection: White
white_coll[white_coll.Name=="circle of protection white", "Name"] = "Circle of Protection: White"

**FIX 3:** _Red_ Collection

In [67]:
red_coll = collections[10]
red_coll.name

'MTGManager/Red Collection'

In [68]:
# RED FIX:
# Garbage Elemental (e) ==> Garbage Elemental
red_coll[red_coll.Name=="Garbage Elemental (e)", "Name"] = "Garbage Elemental"

# Goblin token ==> DROP
goblin_tokens_idx = red_coll[red_coll.Name=="Goblin token card"].index.values
red_coll.drop(goblin_tokens_idx, axis=0, inplace=True)

#### (D) Last check! 

Last double-check that we have all the cards in place, with respect to the Scryfall Oracle

In [69]:
from itertools import filterfalse, chain

def double_check_card_names(c: Collection, oracle: ScryfallDB) -> bool:
    card_names = c.Name.values.tolist()
    f = tuple(filter(lambda cn: cn not in oracle, card_names))
    return not(len(f))

f_double_check = partial(double_check_card_names, oracle=CARDS_ORACLE)

to_recheck = filterfalse(lambda c: c, map(f_double_check, collections))
to_recheck = list(to_recheck)
if not len(to_recheck):
    print("✅ ALL COLLECTIONS OK")
else:
    for c in to_recheck: 
        print(f"{c.name} needs to be still checked")

✅ ALL COLLECTIONS OK


---

### 2.3 Align `ExpansionCode`

What is now needed to do is to align the list of `ExpansionCode` in the _old_ data with the _new_ format. 
In other words, we need to: 
- 1. Re-map ExpansionCode using `EXP_CODES_REPLACE_MAP`
- 2. All cards from Promo Sets (e.g. _FNM, Judge Gifts, Arena Play Network_) will be mapped to the most likely one. 

**Note on 2.**: If multiple possible matches will be found for a single card, the first match will be used. 
<br />
For example:
```python
>>> tuple(CARDS_ORACLE.lookup("Goblin Warchief", set_type="promo"))
(Card(name='Goblin Warchief', lang='en', set_code='f16', 
      set_name='Friday Night Magic 2016', set_type='promo'),
 Card(name='Goblin Warchief', lang='en', set_code='f06', 
      set_name='Friday Night Magic 2006', set_type='promo'))
```

This could potentially induce a difference later on in comparing the collections, but there is so much we can do.

In [70]:
from typing import List 

def align_expansion_codes(coll: Collection, oracle: ScryfallDB, 
                          promo_expansion_list: List[str] = PROMO_EXPANSIONS, 
                          expansion_replace_map: Dict[str, str] = EXP_CODES_REPLACE_MAP,
                          verbose: bool = True) -> None:
    
    # 1. replace expansion codes
    coll.ExpansionCode.replace(expansion_replace_map, inplace=True)

    # 2. look for PROMO expansions in oracle for single cards
    promo_set_codes = [expansion_replace_map[code] for code in promo_expansion_list]
    promo_sets_in_collections = coll.ExpansionCode[coll.ExpansionCode.isin(promo_set_codes)].unique()
    
    if not len(promo_sets_in_collections):
        if verbose:
            print(f"[LOG] No Promo code found in {coll.name} ✅")
            print("="*20, end="\n\n")
        return
    if verbose:
        print(f"[LOG] Matching Codes for Promo Expansions in {coll.name} 👀 🔍")
    for promo_code in promo_sets_in_collections:
        promo_card_names = coll[coll.ExpansionCode == promo_code].Name.unique()
        for card_name in promo_card_names:
            cards_in_db = tuple(oracle.lookup(card_name, set_code=promo_code, set_type="promo"))
            if not cards_in_db:
                print(f"\nNO matching found for {card_name} with Code {promo_code} in {coll.name}.\n")
                continue
            if len(cards_in_db) > 1 and verbose:
                print(f"\nFound more than one candidate for {card_name} in Scryfall. The first will be used.\n")
            new_setcode = cards_in_db[0].set_code
            if verbose: 
                print(f"[LOG] \t ({card_name}, {promo_code}) --> ({card_name}, {new_setcode})")
            coll[((coll.Name == card_name) & (coll.ExpansionCode == promo_code)), "ExpansionCode"] = new_setcode
    if verbose:
        print("="*20, end="\n\n")

In [71]:
align_codes = partial(align_expansion_codes, oracle=CARDS_ORACLE)

_ = list(map(align_codes, collections))

[LOG] No Promo code found in MTGManager/Black Collection ✅

[LOG] Matching Codes for Promo Expansions in MTGManager/Non basic Lands 👀 🔍
[LOG] 	 (Mishra's Factory, g) --> (Mishra's Factory, g05)
[LOG] 	 (Treetop Village, f) --> (Treetop Village, f04)

[LOG] Matching Codes for Promo Expansions in MTGManager/Artifacts 👀 🔍
[LOG] 	 (Icy Manipulator, f) --> (Icy Manipulator, f05)
[LOG] 	 (Isochron Scepter, f) --> (Isochron Scepter, f08)

[LOG] No Promo code found in MTGManager/Green Collection ✅

[LOG] No Promo code found in MTGManager/Islands ✅

[LOG] Matching Codes for Promo Expansions in MTGManager/Gold Multicolor 👀 🔍
[LOG] 	 (Lobotomy, f) --> (Lobotomy, f06)

NO matching found for Naya Sojourners with Code pwp in MTGManager/Gold Multicolor.


[LOG] No Promo code found in MTGManager/Mountains ✅

[LOG] No Promo code found in MTGManager/Swamps ✅

[LOG] Matching Codes for Promo Expansions in MTGManager/White Collection 👀 🔍
[LOG] 	 (Silver Knight, f) --> (Silver Knight, f04)

[LOG] Matching C

In [72]:
tuple(CARDS_ORACLE.lookup("Naya Sojourners", set_type="promo"))

(Card(name='Naya Sojourners', lang='en', set_code='pm10', set_name='Magic 2010 Promos', set_type='promo'),)

⚠️ 

**SO**: Everything has been automatically fixed but **ONE** entry in _Gold_Multicolor_ collection due to a mismatch in _old_ App data.

<ins>This will be fixed _manually_. </ins>

In [73]:
gold_multicolours[((gold_multicolours.Name=="Naya Sojourners") & 
                   (gold_multicolours.ExpansionCode == "pwp")), "ExpansionCode"] = "pm10"

In [74]:
gold_multicolours[gold_multicolours.Name=="Naya Sojourners"]

,Quantity,Name,ExpansionCode,PurchasePrice,Foil,Condition,Language,PurchaseDate
486,2,Naya Sojourners,pm10,0.0,False,NearMint,English,2020-05-31
487,2,Naya Sojourners,arb,0.0,False,NearMint,Italian,2020-05-02


---

### 2.4 Add `ExpansionName` column in the _old_ data format

(Now that all `ExpansionCode` are now aligned with **Scryfall**)

In [75]:
def add_ExpansionName_column(coll: Collection, oracle: ScryfallDB, verbose: bool = True) -> None:
    coll["ExpansionName"] = coll.ExpansionCode.map(oracle.expansion_codename_map)
    # Re-sort columns so to keep the same order as in the NEW data format
    coll.resort_colunns(["Quantity", "Name", "ExpansionCode", "ExpansionName", 
                         "PurchasePrice", "Foil", "Condition", "Language", 
                         "PurchaseDate"])
    if verbose:
        print(f"[LOG] ExpansionName remapepd for {coll.name} ✅")
        print("="*20, end="\n\n")

In [76]:
add_expansioname_f = partial(add_ExpansionName_column, oracle=CARDS_ORACLE, verbose=True)

_ = list(map(add_expansioname_f, collections[:13]))

[LOG] ExpansionName remapepd for MTGManager/Black Collection ✅

[LOG] ExpansionName remapepd for MTGManager/Non basic Lands ✅

[LOG] ExpansionName remapepd for MTGManager/Artifacts ✅

[LOG] ExpansionName remapepd for MTGManager/Green Collection ✅

[LOG] ExpansionName remapepd for MTGManager/Islands ✅

[LOG] ExpansionName remapepd for MTGManager/Gold Multicolor ✅

[LOG] ExpansionName remapepd for MTGManager/Mountains ✅

[LOG] ExpansionName remapepd for MTGManager/Swamps ✅

[LOG] ExpansionName remapepd for MTGManager/White Collection ✅

[LOG] ExpansionName remapepd for MTGManager/Blue Collection ✅

[LOG] ExpansionName remapepd for MTGManager/Red Collection ✅

[LOG] ExpansionName remapepd for MTGManager/Forests ✅

[LOG] ExpansionName remapepd for MTGManager/Plains ✅



In [77]:
collections[0].head()

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
0,4,Abnormal Endurance,m19,Core Set 2019,0.02,False,NearMint,English,2018-08-18
1,2,Absorb Vis,ddk,Duel Decks: Sorin vs. Tibalt,0.07,False,NearMint,English,2018-07-29
2,1,Abyssal Horror,usg,Urza's Saga,0.22,False,NearMint,English,2018-07-29
3,1,Acolyte of Xathrid,m10,Magic 2010,0.05,False,NearMint,English,2018-07-29
4,1,Aphetto Dredging,h09,Premium Deck Series: Slivers,0.00,False,NearMint,English,2018-07-29


In [78]:
collections[13].head()

,Quantity,Name,ExpansionCode,ExpansionName,PurchasePrice,Foil,Condition,Language,PurchaseDate
0,4,Abnormal Endurance,m19,Core Set 2019,0.02,False,NearMint,English,2018-08-18
1,2,Absorb Vis,ddk,Duel Decks: Sorin vs. Tibalt,0.07,False,NearMint,English,2018-07-29
2,1,Abyssal Horror,usg,Urza's Saga,0.22,False,NearMint,English,2018-07-29
3,1,Acolyte of Xathrid,m10,Magic 2010,0.05,False,NearMint,English,2018-07-29
4,1,Aphetto Dredging,h09,Premium Deck Series: Slivers,0.00,False,NearMint,English,2018-07-29


---

### 2.5 Save the _updated_ collections 

In [79]:
_ = list(map(lambda c: c.save(target_folder="../updated"), collections))

MTGManager/Black Collection saved in ../updated/Black_Collection.csv
MTGManager/Non basic Lands saved in ../updated/Non_basic_Lands.csv
MTGManager/Artifacts saved in ../updated/Artifacts.csv
MTGManager/Green Collection saved in ../updated/Green_Collection.csv
MTGManager/Islands saved in ../updated/Islands.csv
MTGManager/Gold Multicolor saved in ../updated/Gold_Multicolor.csv
MTGManager/Mountains saved in ../updated/Mountains.csv
MTGManager/Swamps saved in ../updated/Swamps.csv
MTGManager/White Collection saved in ../updated/White_Collection.csv
MTGManager/Blue Collection saved in ../updated/Blue_Collection.csv
MTGManager/Red Collection saved in ../updated/Red_Collection.csv
MTGManager/Forests saved in ../updated/Forests.csv
MTGManager/Plains saved in ../updated/Plains.csv
MTGDragonShield/Black Collection saved in ../updated/Black_Collection.csv
MTGDragonShield/Non basic Lands saved in ../updated/Non_basic_Lands.csv
MTGDragonShield/Artifacts saved in ../updated/Artifacts.csv
MTGDragonSh

---